# Octane modelling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector as sfs
from sklearn.svm import OneClassSVM

'''custom modules'''
from filter_data import Select_descriptors
from process_data import Scalar
from simmilarity import mean_tanimato,Leverage
from train import Model

In [ ]:
df = pd.read_csv(r'C:\Users\zcemg08\Desktop\phys_data\octane_data.csv')

In [ ]:
df.head(5)

### 1. Data preprocessing

In [ ]:
### 1.1 Remove columns with missing values
misssing_val_cols = df.columns[df.isnull().any()]
print('Columns with missing values are = {}'.format(list(misssing_val_cols)))

In [ ]:
### 1.2 Remove columns with irrelevant to modelling values
df_ = df.drop(misssing_val_cols,axis=1)

irrelevant_to_pred_columns = ['PubChem','ron_choice']

df_ = df_.drop(irrelevant_to_pred_columns,axis=1)

In [ ]:
print('final dataset sie = {}'.format(df_.shape))

### 2. Split dataset (standartization must be applied on train set only )

In [ ]:
seed = 42 # Fix random seed to make split reproducible (experiment must be reproducible)

X_train, X_test, y_train, y_test = train_test_split(df_.drop('y',axis=1), df_['y'], test_size=0.2, random_state=seed)

In [ ]:
print('Train dataset size = {}'.format(X_train.shape))
print('Test dataset size = {}'.format(X_test.shape))

In [ ]:
# Normilise features, so varince threshold can be applied
sc         = Scalar('minmax')

# Wrapper feature selector
wrapper    = sfs(SVR(gamma='auto'),
              n_features_to_select=50,
              scoring='neg_mean_squared_error',
              cv=5)

# Applies varience threshold, removes high correlated features, removes higly skewed vars and applies wrapper in the end.

Filter_    = Select_descriptors(0.01,0.95,None,wrapper)

sc.fit(X_train)

In [ ]:
X_train  = Filter_.transform(sc.transform(X_train),y_train)

In [ ]:
X_train.head(5)

In [ ]:
ml_model = Model('SVM',X_train,y_train,120)

model_   = ml_model.build_model()

### Performance on the test set

In [ ]:
# Calculate prediction errors
y_pred_test = model_.predict(sc.transform(X_test)[X_train.columns[1:]])
mae         = np.abs(y_test.values-y_pred_test)

In [ ]:
plt.scatter(y_pred_test,y_test)
plt.plot(np.linspace(0,130,10),np.linspace(0,130,10))
plt.xlabel('predicted')
plt.ylabel('expected')

In [ ]:
data_error = pd.DataFrame(columns=['SMILES','MAE'])
data_error['SMILES'] = X_test.iloc[np.where(mae>20)[0]]['SMILES'].values
data_error['MAE'] = mae[np.where(mae>20)[0]]

In [ ]:
data_error

### Applicability domain

1. OneClassSVM

In [ ]:
mean_inside = []

for nu in np.linspace(0.001,0.99,20):

    clf = OneClassSVM(nu=nu, kernel="rbf", gamma=0.1)
    clf.fit(X_train.values[:,1:])
    label = clf.predict(sc.transform(X_test)[X_train.columns[1:]])
    mean_inside.append(mae[np.where(label>0)[0]].mean())



In [ ]:
plt.plot(np.linspace(0.001,0.99,20),mean_inside)
plt.xlabel('mu')
plt.ylabel('MAE')

2.Tanimato distance

In [ ]:
DM = mean_tanimato
Tanimato_dist = X_test['SMILES'].apply(lambda x: DM(x,X_train)).values

In [ ]:
plt.scatter(Tanimato_dist,mae)
plt.xlabel('Mean tanimato distance')
plt.ylabel('MAE')

In [ ]:
spread_tanimato = Tanimato_dist.max() - Tanimato_dist.min()

mean_inside2 = []
for i in range(1,20):
    upper_bound = Tanimato_dist.max() - i*spread_tanimato/20
    mean_inside2.append(mae[np.where(Tanimato_dist<upper_bound)[0]].mean())

In [ ]:
plt.plot(np.linspace(0,1,19),mean_inside2)
plt.xlabel('Fraction')
plt.ylabel('MAE')

In [ ]:
def box_plot(dist,mae):

    df_  = pd.DataFrame(np.vstack((dist,mae)).T,columns=['dist','MAE'])
    bins = pd.cut(df_.iloc[:,0], list(np.linspace(dist.max(), dist.min(), 5))[::-1])

    fig,ax = plt.subplots()
    ax.boxplot([g[1] for g in df_.groupby(bins)['MAE']])
    ax.set_xticklabels(str(g[0])[1:-1] for g in df_.groupby(bins)['MAE'])
    ax.set_xlabel('distance bins')
    ax.set_ylabel('MAE')

    fig.tight_layout()


In [ ]:
box_plot(Tanimato_dist,mae)

3. Leaverage distance

In [ ]:
x_test_normed = sc.transform(X_test)[X_train.columns].values[:,1:]
lev_dist      = [Leverage(x_test_normed[i,:],X_train.values[:,1:]) for i in range(len(X_test))]

In [ ]:
plt.scatter(lev_dist,mae)
plt.xlabel('Leaverage distance')
plt.ylabel('MAE')

In [ ]:
box_plot(np.array(lev_dist),mae)


4. Isolation forest

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
mean_inside = []

for cont in np.linspace(0.001,0.99,20):

    clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=cont,
                          max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)
    clf.fit(X_train.values[:,1:])
    label = clf.predict(sc.transform(X_test)[X_train.columns[1:]])
    mean_inside.append(mae[np.where(label>0)[0]].mean())

In [ ]:
plt.plot(np.linspace(0.001,0.99,20),mean_inside)
plt.xlabel('contamination %')
plt.ylabel('MAE')